# ScanBot Model

This will take the image resize it , finds boundaries , edges,

In [4]:
import cv2
import numpy as np
import mapper

def mapp(h):
    h = h.reshape((4,2))     #reshaping the target value
    hnew = np.zeros((4,2),dtype = np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h,axis = 1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]

    return hnew

pic=cv2.imread(r'C:\Users\Hp\Downloads\WhatsApp Image 2020-09-19 at 17.07.39.jpeg')   #reads the image from the given dirctory
pic=cv2.resize(pic,(1300,800)) #resizing the image for better functioning
temp=pic.copy()

cnvgray=cv2.cvtColor(pic,cv2.COLOR_BGR2GRAY)  #RGB To Gray Scale
cv2.imshow("Heading",cnvgray)

blur=cv2.GaussianBlur(cnvgray,(5,5),0) #gaussianBlur will smoothen out your image, (5,5) is the kernel size and 0 is sigma that determines the amount of blur
cv2.imshow("Blur",blur)

edge=cv2.Canny(blur,30,50)  #30 MinThreshold and 50 is the MaxThreshold , these are the max, min values which helps to find canny edges 
cv2.imshow("Canny",edge)


contours,hierarchy=cv2.findContours(edge,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)  #retrieve the contours as a list, with simple apprximation model
contours=sorted(contours,key=cv2.contourArea,reverse=True)    #to find the biggest contour

#the loop extracts the boundary of the page
for c in contours:
    s=cv2.arcLength(c,True) #applicable for closed shapes
    approx=cv2.approxPolyDP(c,0.02*s,True)

    if len(approx)==4:      #to check if its square
        target=approx
        break
        
approx=mapp(target)    #find endpoints of the sheet

pts=np.float32([[0,0],[800,0],[800,800],[0,800]])  #map to 800*800 target window

op=cv2.getPerspectiveTransform(approx,pts)  #get the top or bird eye view effect
dst=cv2.warpPerspective(temp,op,(800,800))


cv2.imshow("Scanned",dst)
# press q or Esc to close
cv2.waitKey(0)
cv2.destroyAllWindows()